<h2 style="color:red;text-align:center;font-weight:bold;">AeroStream Analytics</h2>

AeroStream Analytics est un système intelligent de classification automatique des avis clients des compagnies aériennes. Il analyse en temps réel le sentiment des utilisateurs afin de mesurer leur niveau de satisfaction et fournir des indicateurs clés de performance.

**Objectifs :**

Développer un système de classification automatique des avis clients en temps réel, Le
système devra permettre de:

- Collecter et prétraiter les avis clients,

- Analyser automatiquement le sentiment et la satisfaction,

- Générer des indicateurs de performance par compagnie aérienne,

- Visualiser les résultats via un tableau de bord interactif.

![Python](https://img.shields.io/badge/Python-3.9%2B-blue)
![Airflow](https://img.shields.io/badge/Apache%20Airflow-Orchestration-green)
![Streamlit](https://img.shields.io/badge/Streamlit-Dashboard-red)
![ChromaDB](https://img.shields.io/badge/ChromaDB-Vector%20Store-orange)

<br>

<h3 style="color:green;font-weight:bold;">Stockage dans ChromaDB :</h3>

<h4 style="color:orange;font-weight:bold;">1. Charger les Metadonnées :</h4>

In [1]:
import pandas as pd

metadata = pd.read_pickle("../metadata/metadata.pkl")

print("Metadonnées Chargées avec Succès !")

metadata.head()

Metadonnées Chargées avec Succès !


,id,text,embedding,sentiment
0,tweet_0,what said,"[0.04736202210187912, -0.059927985072135925, -...",neutral
1,tweet_1,i didn t today must mean i need to take ano...,"[-0.007803764659911394, 0.004880663473159075, ...",neutral
2,tweet_2,it s really aggressive to blast obnoxious ent...,"[0.013941123150289059, -0.003644852200523019, ...",negative
3,tweet_3,and it s a really big bad thing about it,"[0.03966449573636055, -0.0010603367118164897, ...",negative
4,tweet_4,seriously would pay 30 a flight for seats tha...,"[0.0020552556961774826, 0.05253976956009865, -...",negative


<h4 style="color:orange;font-weight:bold;">2. ChromaDB :</h4>

<h5 style="font-weight:bold;">2.1. Vérifier l'installation de ChromaDB :</h5>

In [2]:
import chromadb

print("Version :", chromadb.__version__)

Version : 1.3.6


<h5 style="font-weight:bold;">2.2. Créer la Collection :</h5>

In [3]:
client = chromadb.PersistentClient(path="../chroma_db")

collection = client.create_collection("data", metadata={"hsnw:space" : "cosine"})

<h5 style="font-weight:bold;">2.3. Insérer Embeddings, Identifiants et Labels  :</h5>

In [4]:
from tqdm import tqdm

batch_size = 5000
total_items = len(metadata)

for i in tqdm(range(0, total_items, batch_size), desc="Insertion dans ChromaDB"):
    batch = metadata.iloc[i:i + batch_size]

    batch_ids = batch["id"].astype(str).tolist()
    batch_embeddings = batch["embedding"].tolist()
    batch_metadata = batch[["sentiment", "text"]].to_dict("records")

    collection.add(
        ids=batch_ids,
        embeddings=batch_embeddings,
        metadatas=batch_metadata
    )

print(f"- Nombre Total d'items dans la Collection : {collection.count()}")

Insertion dans ChromaDB: 100%|██████████| 3/3 [00:06<00:00,  2.09s/it]

- Nombre Total d'items dans la Collection : 14220


In [5]:
item = collection.peek(limit=1)

print("\nTrain Item : ")
print(f"- ID: {item['ids'][0]}")
print(f"- Label: {item['metadatas'][0]['sentiment']}")
print(f"- Text: {item['metadatas'][0]['text'][:50]}...")
print(f"- Embedding : {item['embeddings'][0][:10]}...")
print(f"- Embedding length: {len(item['embeddings'][0])}")


Train Item : 
- ID: tweet_0
- Label: neutral
- Text: what said ...
- Embedding : [ 0.04736202 -0.05992799 -0.00408231 -0.02175779  0.03396935 -0.0464993
 -0.00246478  0.00842592 -0.02412003  0.02323108]...
- Embedding length: 768
